## Task 2.1
Consists of extracting data from the dictionary and adding the same in a nested dictionary & storing them as JSON files

In [ ]:
from bs4 import BeautifulSoup
import os 
import re
import nltk
from copy import deepcopy
import json

In [ ]:
ECTNestedDict = {}
transcript = {}
file_path = os.getcwd() + "/ECT/"

script_file_path = os.getcwd() + "/ECTNestedDict/"
os.mkdir(script_file_path)
file_list = os.listdir(file_path)
debug_list = []

# Dictionary Construction
for i in range(len(file_list)):
    path = file_path + file_list[i]
    ID = re.findall(r'\d+', file_list[i])[0]
    # print("DocID: " + ID)

    tr_file = open(path)
    # Parsing the html
    soup = BeautifulSoup(tr_file, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    #  --------------------------------------------------------------------------------------------------------------
    # Finding & Storing Date
    # Storing the 1st line
    a = (soup.find('p', {'class' : 'p p1'})).get_text()  # For finding a specific class in the html
    #  Finding the date using regex
    a = re.findall(r'[A-Z]\w+\s+\d+\,\s+\d+',a)

    
    # Brute-Forcing the "p p_1" class in case Dat not in heading
    if(not a): 
        temp = soup.findAll('p', class_="p p1")
        for j in range(5): # Checking in the 1st 5 tags
            a = re.findall(r'[A-Z]\w+\s+\d+\,\s+\d+', temp[j].get_text())
            if(not a):
                a = re.findall(r'[A-Z]\w+\s+\d+\s+\d+', temp[j].get_text())
            if(a):
                break
    if(not a):
        print(temp[j].get_text())
    # Removing extra spaces from the date
    date = " ".join(a[0].split())

    #  --------------------------------------------------------------------------------------------------------------
    # Constructing Participant List
    a = soup.findAll('p')
    part_list= []
    flag=0  # flag for storing no. of strong texts <strong> texts
    ctr=-1

    while(flag<3):
        ctr+=1
        tag = a[ctr]
        if(tag.find('strong')):
            flag+=1
        elif(flag>0):
            part_list.append(tag.get_text())

    #  --------------------------------------------------------------------------------------------------------------
    # Constructing Presentation Dictionary
    presentation_dict ={}
    temp =[]
    speaker = ""
    flag = 0
    qna_head_phrases = ["question-and-answer session", "question-and-answers session", "questions-and-answers session", "questions-and-answer session"]

    while(ctr<len(a) and (" ".join(((a[ctr].get_text()).lower()).split()) not in qna_head_phrases)):
        tag = a[ctr]
        if(tag.find('strong')):
            if((tag.get_text() not in presentation_dict.keys())):
                if(flag==0):
                    speaker = tag.get_text()
                    flag=1
                elif(flag!=0):
                    presentation_dict[speaker] = temp
                    temp=[]
                    speaker = tag.get_text()
            else:
                presentation_dict[speaker] = temp
                speaker = tag.get_text()
                temp = presentation_dict[speaker]
        else:
            temp.append(tag.get_text())
        ctr+=1
    else: # To add the last speaker to presentation_dict
        presentation_dict[speaker] = temp
    
    if(ctr==len(a)):
        debug_list.append(ID)

    #  --------------------------------------------------------------------------------------------------------------
    # Construction of Questionnaire Dictionary
    ctr+=1 # To move forward from the Q-&-A Heading tag
    question_dict = {}
    temp = {}
    speaker = ""
    flag = 0
    remarks = []
    flow_ctr= 1
    while(ctr<len(a)): # Looping till end of the document
        tag =a[ctr]
        if((tag.find('strong') or (tag.get_text() in part_list))):
            if(flag==0):
                speaker = tag.get_text()
                flag=1
            else:
                temp["Speaker"] = deepcopy(speaker)
                temp["Remarks"] = remarks
                question_dict[flow_ctr] = deepcopy(temp) # Important to use deepcopy
                speaker = tag.get_text()
                remarks = []
                temp = {}
                flow_ctr+=1
        else:
            remarks.append(tag.get_text())
        ctr+=1
    else: # To add the last speaker to question_dict. 
        temp["Speaker"] = deepcopy(speaker)
        temp["Remarks"] = remarks
        question_dict[flow_ctr] = deepcopy(temp)

    #  --------------------------------------------------------------------------------------------------------------    
    # Creating the Transcript Instance 
    transcript["Date"] = date
    transcript["Presentation"] = presentation_dict
    transcript["Questionnaire"] = question_dict
    # Adding the transcript to overall nested dictionary
    ECTNestedDict["Transcript-" + str(i+1)] = deepcopy(transcript)

    #  --------------------------------------------------------------------------------------------------------------    
    # Saving the transcript Dictionary in a JSON file
    script_file_path = os.getcwd() + "/ECTNestedDict/"
    json_name = "transcript-" + str(i+1) + ".json"
    with open(script_file_path + json_name, 'w') as fp:
        json.dump(transcript, fp, sort_keys=True, indent=3)

In [ ]:
# Saving the entire transcripts in a big JSON.
script_file_path = os.getcwd() + "/"
json_name = "ECTNestedDict.json"
with open(script_file_path + json_name, 'w') as fp:
    json.dump(ECTNestedDict, fp, sort_keys=True, indent=3)

In [ ]:
# Debug for Task 2.1 (Single-Instance)
print("DocID: " + ID)
print("Date: " + date)

print("----------------------------------------------------------------------------------------------------------------------------------------------")

print("PARTICIPANT LIST")
for temp in part_list:
    print(temp)

print("----------------------------------------------------------------------------------------------------------------------------------------------")
print("PRESENTATION")
for temp in presentation_dict:
    print("Speaker: " + temp)
    for b in presentation_dict[temp]:
        print(b)

print("----------------------------------------------------------------------------------------------------------------------------------------------")
print("QUESTIONNAIRE ")
for i in range(1, len(question_dict.keys())+1):
    temp = question_dict[i]
    print(str(i) + ": " + temp["Speaker"])
    for b in temp["Remarks"]:
        print(b)

## Task 2.2
Extracting only the text files from **ECTNestedDict** & store in **ECTText**

In [ ]:
# JSON file location
file_path = os.getcwd() + "/ECTNestedDict/"
# Creating "ECTText" folder
txt_file_path = os.getcwd() + "/ECTText/"
os.mkdir(txt_file_path)
file_list = os.listdir(file_path)
debug_list = []

for i in range(1,len(file_list)+1):
    # JSON + TXT file name for Read/Write 
    json_file_name = file_path + "transcript-" + str(i) + ".json"
    txt_file_name = txt_file_path + "transcript-" + str(i) + ".txt"
    text = ""

    # Opening JSON file 
    with open(json_file_name, 'r') as transcript_file:
        transcript = json.load(transcript_file)
        presentation_dict = transcript["Presentation"]
        for presenter in presentation_dict:
            remarks = presentation_dict[presenter]
            if(remarks):
                text = text + remarks[0]
            for j in range(1, len(remarks)):
                text = text + " " + remarks[j]
        question_dict = transcript["Questionnaire"]
        for speaker in question_dict:
            remarks = question_dict[speaker]["Remarks"]
            for temp in remarks:
                text = text + " " + temp
    with open(txt_file_name,"wt") as text_file:
        text_file.write(text)


## Task 3

### Task 3.1
- Tokenization
- Lemmatization
- Stop-Words/Punctuations Removal


### Task 3.2
- Build Inverted Positional Index

#### **HINT-** Positional Index Structure

![Example Posting Index](img/1.png)

For the current example this what our Posting-List structure would look like:

```
{"hello" : { 3 : [120, 125, 278], 
             5 : [28] , 
             10 : [132, 182], 
             23 : [0, 12, 28], 
             27 : [2] 
           } }
```


In [32]:
# import numpy as np 
import os 
import nltk 
from nltk.corpus import stopwords
from copy import deepcopy
import json

# Tokenizer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [41]:
txt_file_path = os.getcwd() + "/ECTText/"
stop_words = set(stopwords.words('english'))
# cust_stop_words = ["'s"]

# Adding custom words to the 
for temp in cust_stop_words:
    stop_words.add(temp)

inv_pos_index = {}

i=1
txt_file_name = txt_file_path + "transcript-" + str(i) + ".txt"
filtered_tokens = [] 
with open(txt_file_name,"r") as text_file:
    text = text_file.read()
    # Tokenizing
    tokens = tokenizer.tokenize(text.lower())
    # Lemmatization
    # TODO
    # -----------------------------------------------------------------------------------------------------------
    for temp in tokens:
        temp1 = lemmatizer.lemmatize(temp)
        # if(temp1!= temp):
        #     print(temp + " | " + temp1)
    # -----------------------------------------------------------------------------------------------------------
    
    # Stop-Word Removal
    for j in range(len(tokens)):
        temp = tokens[j]
        if(temp not in stop_words):
            filtered_tokens.append((temp,j)) # Storing their position in the original document as well for easier search
    
    #  Building Corpus
    for token in filtered_tokens:
        if(token[0] not in inv_pos_index):
            token_dict = {}
            pos_list = [token[1]]
            token_dict[i] = pos_list
            inv_pos_index[token[0]] = deepcopy(token_dict)
        
        else:
            token_dict = inv_pos_index[token[0]]
            if(i not in token_dict):
                pos_list = [token[1]]
                token_dict[i] = pos_list
                inv_pos_index[token[0]] = deepcopy(token_dict)
            else:
                token_dict[i].append(token[1])


    print(inv_pos_index)

{&#39;thank&#39;: {1: [0, 232, 2557, 2575, 2607, 2835, 3706, 4445, 4895, 5396, 6121, 6720, 6730, 7310, 7320, 7352, 7375, 7395, 7406]}, &#39;good&#39;: {1: [2, 228, 3289, 3655, 3745, 3788, 4139, 4466, 5035, 5106, 5112, 5820, 6347, 6727, 6761, 7050, 7076, 7203, 7393]}, &#39;morning&#39;: {1: [3, 229]}, &#39;everyone&#39;: {1: [4, 230]}, &#39;happy&#39;: {1: [7, 1456]}, &#39;welcome&#39;: {1: [9, 2613]}, &#39;vista&#39;: {1: [12, 25, 34, 161, 197, 2580, 2615, 4967, 6952, 7171, 7362]}, &#39;second&#39;: {1: [14, 649, 918, 1070, 1170, 1337, 2618, 2701, 2715, 2785, 3375, 3379, 3638, 4323, 5327, 6084, 6300, 6420, 6479]}, &#39;quarter&#39;: {1: [15, 380, 486, 643, 650, 701, 717, 739, 780, 907, 919, 1033, 1171, 1194, 1206, 1281, 1296, 1328, 1357, 1477, 1497, 2317, 2370, 2619, 2670, 2918, 2935, 2979, 4396, 6598, 6689, 6737, 7389]}, &#39;2020&#39;: {1: [16, 754, 1439, 1582, 1629, 1647, 1684, 2328, 2377, 2620, 2912, 4056, 7149]}, &#39;results&#39;: {1: [17, 86, 332, 1936, 4397, 4987]}, &#39;call&#

a
